# Custom Model Training

In [1]:
# Use the Azure Machine Learning data collector to log various metrics
from azureml.logging import get_azureml_logger
logger = get_azureml_logger()

In [2]:
# Use Azure Machine Learning history magic to control history collection
# History is off by default, options are "on", "off", or "show"
# %azureml history on

In [3]:
import sys, os, os.path, glob
# To ensure reproducibility, we need to follow these steps
# as explained in https://keras.io/getting-started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development
os.environ['PYTHONHASHSEED'] = '0'
keras_backend = 'cntk' # valid values: cntk, tensorflow
os.environ['KERAS_BACKEND'] = keras_backend

seed = 1343
import numpy as np
np.random.seed(seed)
import random as rn
rn.seed(seed)

if keras_backend == 'tensorflow':
    import tensorflow as tf

    # Force TensorFlow to use single thread.
    # Multiple threads are a potential source of
    # non-reproducible results.
    # For further details, see: https://stackoverflow.com/questions/42022950/which-seeds-have-to-be-set-where-to-realize-100-reproducibility-of-training-res
    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

from keras import backend as K
K.set_learning_phase(False)

if keras_backend == 'tensorflow':
    tf.set_random_seed(seed)

    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    K.set_session(sess)

if keras_backend == 'cntk':
    import cntk as C
    C.device.try_set_default_device(C.device.gpu(0))
    
from keras.applications import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.models import Model


Using CNTK backend


In [4]:
IM_WIDTH, IM_HEIGHT, IM_CHANNELS = 224, 224, 3 #fixed size for InceptionV3

def setup_dnn():
    #K.set_image_data_format('channels_last')
    model = InceptionV3(weights="imagenet", include_top=False, input_shape=(IM_WIDTH, IM_HEIGHT, IM_CHANNELS)) #include_top=False excludes final FC layer
    return model

def setup_transfer_learninig(model, base_model):
    """Freeze all layers and compile the model"""
    for layer in base_model.layers:
        layer.trainable = False  
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

def setup_new_classifier(base_model, nb_classes):
    """Add last layer to the convnet
        Args:
            base_model: keras model excluding top
            nb_classes: # of classes
        Returns:
            new keras model with last layer
    """
    FC_SIZE = 1024
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(FC_SIZE, activation='relu')(x) #new FC layer, random init
    predictions = Dense(nb_classes, activation='softmax')(x) #new softmax layer
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

def setup_finetune(model):
    """Freeze the bottom NB_IV3_LAYERS and retrain the remaining top layers.
        note: NB_IV3_LAYERS corresponds to the top 2 inception blocks in the inceptionv3 arch
        Args:
            model: keras model
    """
    NB_IV3_LAYERS_TO_FREEZE = 172
    
    for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
        layer.trainable = False
    for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
        layer.trainable = True
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
    
def save_dnn(model, folder, filename):
    filepath = os.path.join(folder, filename)
    sess = K.get_session()
    
    outputs = ["input_1", "dense_2/Softmax"]
    constant_graph = tf.graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), outputs)
    tf.train.write_graph(constant_graph,folder,filename,as_text=False)
    #tf.train.write_graph(sess.graph.as_graph_def(), folder, filename, as_text=False)
    #tf.train.export_meta_graph()
    print('saved the graph definition in tensorflow format at: ', filepath)

def utils_files_count(directory):
    """Get number of files by searching directory recursively"""
    if not os.path.exists(directory):
        return 0
    cnt = 0
    for r, dirs, files in os.walk(directory):
        for dr in dirs:
            cnt += len(glob.glob(os.path.join(r, dr + "/*")))
    return cnt

def train_generator(folder, batch_size=16, save_to_dir=None):
    datagen =  ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )
    #generated_folder = os.path.join(folder, '..', 'generated')
    if save_to_dir:
        if not os.path.exists(save_to_dir):
            os.makedirs(save_to_dir)
        else:
            utils_removeFilesInFolder(save_to_dir)

    generator = datagen.flow_from_directory(
        folder,
        target_size=(IM_WIDTH, IM_HEIGHT),
        batch_size=batch_size,
        save_to_dir=save_to_dir
    )
    return generator

def validation_generator(folder, batch_size=16):
    #IM_WIDTH, IM_HEIGHT = 299, 299 #fixed size for InceptionV3
    datagen =  ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rotation_range=45,
        width_shift_range=0,
        height_shift_range=0,
        shear_range=0,
        zoom_range=0,
        horizontal_flip=True
    )
    
    generator = datagen.flow_from_directory(
        folder,
        target_size=(IM_WIDTH, IM_HEIGHT),
        batch_size=batch_size
    )
    return generator

def utils_removeFilesInFolder(folder):
    files = glob.glob(os.path.join(folder,'*'))
    for f in files:
        os.remove(f)

def utils_generated_sample (data_folder, generated_folder, batches_length=1):
    generator = train_generator(data_folder, batch_size=16, save_to_dir=generated_folder)
    
    i = 0
    for batch in generator:
        i += 1
        if i > batches_length:
            break  # otherwise the generator would loop indefinitely

def save_cntk(model, folder, filename):
    import cntk as C
    C.combine(model.outputs).save(os.path.join(folder, filename))
    

In [5]:
project_folder = %pwd
print("Project folder: ", project_folder)
output_folder = os.path.join(project_folder,'outputs')
data_folder = os.path.join(project_folder, 'data')
train_folder = os.path.join(data_folder, 'train')
validation_folder = os.path.join(data_folder, 'validation')
nb_classes = len(glob.glob(train_folder + "/*"))
batch_size=16
nb_epoch = 32
# load weights
model_filename = 'model.caret.pb'
#if not os.path.exists(output_folder):
#    os.makedirs(output_folder)

Project folder:  D:\fran\Work\eShoConAI\workbench


In [6]:
# setup model
base_model = setup_dnn()
model      = setup_new_classifier(base_model, nb_classes)

# transfer learning
setup_transfer_learninig(model, base_model)

history_tl = model.fit_generator(
    train_generator(train_folder,batch_size),
    steps_per_epoch=500//batch_size, #utils_files_count(train_folder)//batch_size,
    epochs=32,
    validation_data=validation_generator(validation_folder, batch_size),
    validation_steps=100//batch_size, #utils_files_count(validation_folder)//batch_size,
    verbose=1,
    class_weight='auto')

D:\fran\AppData\local\AmlWorkbench\Python\lib\site-packages\keras\backend\cntk_backend.py:2337: UserWarning: CNTK backend warning: CNTK version not detected. Will using CNTK 2.0 GA as default.
  'CNTK backend warning: CNTK version not detected. '


Found 72 images belonging to 4 classes.
Found 15 images belonging to 4 classes.
Epoch 1/32
31/31 [==============================] - 58s 2s/step - loss: 3.5321 - acc: 0.6778 - val_loss: 2.9672 - val_acc: 0.7444
Epoch 2/32
31/31 [==============================] - 42s 1s/step - loss: 0.7410 - acc: 0.8247 - val_loss: 0.3864 - val_acc: 0.8667
Epoch 3/32
31/31 [==============================] - 43s 1s/step - loss: 0.3852 - acc: 0.8891 - val_loss: 1.0535 - val_acc: 0.8111
Epoch 4/32
31/31 [==============================] - 44s 1s/step - loss: 0.2642 - acc: 0.9214 - val_loss: 0.4107 - val_acc: 0.9000
Epoch 5/32
31/31 [==============================] - 40s 1s/step - loss: 0.5472 - acc: 0.9092 - val_loss: 0.5422 - val_acc: 0.8556
Epoch 6/32
31/31 [==============================] - 41s 1s/step - loss: 0.3123 - acc: 0.9474 - val_loss: 1.3453 - val_acc: 0.8556
Epoch 7/32
31/31 [==============================] - 40s 1s/step - loss: 0.2112 - acc: 0.9358 - val_loss: 2.3539 - val_acc: 0.8333
Epoch 8/32

In [ ]:
#save_dnn(model, output_folder, model_filename)
save_cntk(model, output_folder, 'model_cntk.pb')

In [ ]:
#utils_generated_sample(train_folder, os.path.join(data_folder, 'generated'),2)

In [ ]:
#model.get_weights()

In [ ]:
print(history_tl.history.keys())

In [ ]:
import numpy
print ("Accuracy: ", numpy.mean(history_tl.history['acc']))

In [ ]:
labels = ["bracelet","earrings","parasol","thermometer"]
def load_image(img_path):
    from keras.preprocessing import image
    #IM_WIDTH, IM_HEIGHT = 299, 299 #fixed size for InceptionV3
    img = image.load_img(img_path, target_size=(IM_WIDTH, IM_HEIGHT))
    x = image.img_to_array(img)
    x = numpy.expand_dims(x, axis=0)
    # https://github.com/fchollet/keras/blob/master/keras/applications/imagenet_utils.py
    # Image Transformation: channels_last & apply zero-center by mean  pixel: [103.939,116.779,123.68]
    x = preprocess_input(x)
    return x

In [ ]:
bracelet_path = os.path.join(validation_folder,'bracelet','3506.jpg')
bracelet_image = load_image(bracelet_path)
preds = model.predict(bracelet_image)
print (preds)
print (labels[preds.argmax()])

In [ ]:
earrings_path = os.path.join(validation_folder,'earrings','3451b.jpg')
earrings_image = load_image(earrings_path)
preds = model.predict(earrings_image)
print (preds)
print (labels[preds.argmax()])

In [ ]:
parasol_path = os.path.join(validation_folder,'parasol','114a.jpg')
parasol_image = load_image(parasol_path)
preds = model.predict(parasol_image)
print (preds)
print (labels[preds.argmax()])

In [ ]:
thermo_path = os.path.join(validation_folder,'thermometer','1866a.jpg')
thermo_image = load_image(thermo_path)
preds = model.predict(thermo_image)
print (preds)
print (labels[preds.argmax()])

In [ ]:
def save_dnn2(model, folder, filename):
    filepath = os.path.join(folder, filename)
    sess = K.get_session()
    graph1 = sess.graph.as_graph_def()
    vars = {}
    for v in tf.trainable_variables():
        vars[v.value().name] = sess.run(v)
    
    with tf.Graph().as_default() as graph2:
        consts = {}
        for k in vars.keys():
            consts[k] = tf.constant(vars[k])
        tf.import_graph_def(graph1,input_map={name:consts[name] for name in consts.keys()})
        
        tf.train.write_graph(sess.graph_def,folder,filename,False)
        #tf.train.write_graph(sess.graph.as_graph_def(), folder, filename, as_text=False)
        print('saved the graph definition in tensorflow format at: ', filepath)

In [ ]:
#save_dnn2(model, output_folder, 'model2.pb')

In [ ]:
print (model.input.name)
print (model.output.name)

In [ ]:
def save_dnn3(model, folder, filename):
    filepath = os.path.join(folder, filename)
    sess = K.get_session()
    graph = sess.graph.as_graph_def()
    
    outputs = ["dense_2/Softmax"]
    constant_graph = tf.graph_util.convert_variables_to_constants(sess, graph, outputs)
    tf.train.write_graph(constant_graph,folder,filename,False)

In [ ]:
#save_dnn3(model, output_folder, 'model3.pb')

In [ ]:
#model.get_output_at("batch_normalization_1/keras_learning_phase")

In [ ]:
model.summary()

In [ ]:
#save_dnn(model, output_folder, model_filename)

In [ ]:
thermo_image.shape

In [ ]:
K.image_data_format()

In [ ]:
model.input.shape

In [ ]:
model.output.shape

In [ ]:
#K.set_image_data_format('channels_first')
K.image_data_format()

In [ ]:
import cntk as C
print(C.device.all_devices())
dev_gpu = C.device.all_devices()[0]
print(dev_gpu.type)
print(C.device.gpu(0))
print(C.device.get_gpu_properties(dev_gpu))
C.device.try_set_default_device(C.device.gpu(0))

In [ ]:
C.device.try_set_default_device

In [ ]:
'%.5f' % (-1)

In [1]:
from azureml.api.schema.dataTypes import DataTypes
DataTypes.STANDARD